In [ ]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
import allantools as at
from typing import Callable, Iterable, Optional, Tuple
from functools import partial
import functools
import inspect
from runData import runData

In [ ]:
pd.set_option("display.precision", 15)      # Show up to 15 decimal places
dirName  = 'labData/'
#rowLimit = 2000 # Rows to keep after joining ticc and timTp samples

In [ ]:
# Big Picture
#   Read TICC data from runs of interest
#   Merge TICC channels chA and chB by time
#   Process TICC data, do simple analysis without timTp data
#   Read timTp data
#   Merge timTp channels chA and chB by time
#   Merge TICC and timTp data by time
#   Plot and analyze results

## Read TICC Data

In [ ]:
# Read TAPR TICC data into a dataframe, as captured to a file by ticc.py running on a host.
# Events are rising edges of a PPS signal from a DUT, resulting in a timestamp on the TICC's reference clock.
# Columns:
#  ppsHostClock: Host clock when serial data for PPS event timestamp was read (ticc.py logs this in UTC)
#  ppsRefClock:  Reference clock when PPS event happened (elapsed seconds since TICC started)
#
# The frequency of the TICC reference clock comes from an external 10 MHz source, a Geppetto Electronics GNSSDO in my case.
# It should be almost exactly 1e7 times the PPS frequency, with an arbitray phase relationship.
# So we expect the whole number portion of ppsRefClock to increment by 1 every second, while the fractional seconds jitter around
# some slowly-changing phase offset.
# Therefore, there's very little information in the whole seconds, while the fractional seconds contain the most interesting data.
# And as the whole number grows with a floating point representation, precision is lost in the fractional digits.
# So once we confirm the whole number of seconds is behaving as expected, we can drop it and focus on the fractional seconds.
# Whatever slowly-changing phase offset exists, it won't impact the deviation metrics.
# We treat the fractional seconds as an instantaneous (but nosiy) measurement of the phase error against the ref clock.
#
# There is a very small, but non-zero chance that the static phase offset plus the jitter causes sequential PPS timestamps to be
# within the same second or more than one second apart, leading to missing or duplicate whole seconds.
# Instead of properly handling whole seconds when this happens, just fail on assertions.
def readTicc(baseName, chan):
    ticcFile = f"{dirName}/{baseName}.ticc{chan}.csv"
    ticcData = pd.read_csv(ticcFile, dtype={'ppsHostClock': str, 'ppsRefClock': str})

    # Convert host timestamp string to UTC timestamp
    ticcData["ppsHostClock"] = pd.to_datetime(ticcData.ppsHostClock, utc=True)

    # Assuming host clock sync is better than serialization latency of timestamp arriving,
    # floor of host clock second will be the navigation epoch sencond.
    # Will be used for later join with TIM-TP timestamps.
    ticcData["epochSec"] = ticcData["ppsHostClock"].dt.floor("s")


    # Split ppsRefClock string into whole and fractional seconds
    ticcData[["rcWhole", "rcFrac"]] = ticcData.ppsRefClock.str.split(".", n=1, expand=True)

    # Check for missing or duplicate whole seconds
    ticcData['rcWhole'] = ticcData['rcWhole'].astype(int)
    expected = set(range(ticcData.rcWhole.min(), ticcData.rcWhole.max() + 1))
    observed = set(ticcData.rcWhole)
    missing = sorted(expected - observed)
    duplicates = ticcData.rcWhole[ticcData.rcWhole.duplicated()].unique().tolist()
    assert len(missing)    == 0, f"Missing whole seconds in ticc{chan}: {missing}"
    assert len(duplicates) == 0, f"Duplicate whole seconds in ticc{chan}: {duplicates}"

    # Convert ref clock fractional part from digit string to float
    ticcData['rcFrac'] = "0." + ticcData['rcFrac'].astype(str)
    ticcData['rcFrac'] = ticcData['rcFrac'].astype(float)

    # Also get fractional part of host clock
    ticcData['hcFrac'] = (ticcData.ppsHostClock.astype('int64')-1e9*(ticcData.ppsHostClock.astype('int64')//1e9))/1e9

    # With overly careful consideration of maintining floating point precision, get interval between PPS events on ref clock.
    ticcData['rcTi'] = (ticcData.rcWhole-ticcData.rcWhole.shift(1)) + (ticcData.rcFrac - ticcData.rcFrac.shift(1)) # Time interval between refClock samples on ref clock

    ticcData['bn'  ] = baseName
    ticcData['dut' ] = runData[baseName][chan]
    ticcData['chan'] = chan
    return ticcData

In [ ]:
dfs = []

dfs.append(readTicc('baseline1', 'A'))
dfs.append(readTicc('baseline2', 'A'))
dfs.append(readTicc('baseline3', 'A'))
dfs.append(readTicc('baseline4', 'A'))
dfs.append(readTicc('baseline5', 'A'))

dfs.append(readTicc('fixedPos1', 'A'))
dfs.append(readTicc('fixedPos2', 'A'))

dfs.append(readTicc('fixedL1ca1', 'A'))
dfs.append(readTicc('fixedL1ca2', 'A'))

dfs.append(readTicc('fixedL1l51', 'A'))
dfs.append(readTicc('fixedL1l52', 'A'))
dfs.append(readTicc('fixedL1l52', 'B'))
dfs.append(readTicc('fixedL1l53', 'A'))
dfs.append(readTicc('fixedL1l53', 'B'))

dfs.append(readTicc('f9tM600-1', 'A'))
dfs.append(readTicc('f9tM600-1', 'B'))
dfs.append(readTicc('f9tM600-2', 'A'))
dfs.append(readTicc('f9tM600-2', 'B'))

dfs.append(readTicc('ctiCns1', 'A'))
dfs.append(readTicc('ctiCns1', 'B'))

ticc = pd.concat(dfs, ignore_index=True)

## Merge TICC Channels chA and chB Data by Time (Reference Clock Second)

In [ ]:
# Bring data from TICC channels for each run together for each ref clock second.
# Retain epoch second for later join
ticcA = ticc[ticc.chan == 'A'][['bn', 'rcWhole', 'rcFrac', 'epochSec']]
ticcB = ticc[ticc.chan == 'B'][['bn', 'rcWhole', 'rcFrac'            ]]

# Perform inner join on bn and rcWhole
rcSec = pd.merge(ticcA, ticcB, on=['bn', 'rcWhole'], how='outer', suffixes=('A', 'B'))

# Derive columns of interest from the TICC data.
if ticcB.empty:
    rcSec.drop(columns=['rcFracB'], inplace=True)
else:
    rcSec['rcFracAB'] = rcSec.rcFracA - rcSec.rcFracB  # Phase difference between the two channels

## Do Simple TICC Data Analysis Without TIM-TP Data

In [ ]:
# XXX Fixme when run with no channel B
beg = 1000
end = 2000
bn = 'fixedL1l53'
plt.figure(figsize=(24, 6))
plt.plot(rcSec[rcSec.bn==bn].epochSec[beg:end], rcSec[rcSec.bn==bn].rcFracAB[beg:end]*1e9, marker='.', linestyle='-', color='b')
plt.title(f"Phase Difference (rcFracAB) vs Epoch Second for Run {bn}")
plt.xlabel('Epoch Second')
plt.ylabel('Phase Difference (rcFracAB)')
plt.grid()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(22, 6))
fig.canvas.draw()
bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
widthPx = int(bbox.width*fig.dpi)
print("Figure width in pixels:", widthPx)
print(fig.get_figwidth())
for bn in ticc.bn.unique():
    ticcSubset = ticc[ticc.bn == bn]
    plt.hist(1e9*(ticcSubset.rcTi-1.0), bins=widthPx, label=bn, alpha=0.5)
#plt.hist(1e9*(ticc[ticc.bn==bn].rcTi[beg:end]-1.0), bins=widthPx, color='blue', alpha=0.7)
plt.title('Histogram of Time Error Between PPS Pulses')
plt.xlabel('Time Error (ns)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:g} ns"))
plt.show()



## Read TIM-TP Data

In [ ]:
def readTimTp(baseName, chan):
    timTp = pd.read_csv(f"{dirName}/{baseName}.timTp{chan}.csv")

    # Confirm expected values in constant columns, then drop them
    assert (timTp['timeBase'   ] ==  1).all(), "Not all rows in timTp.timeBase are equal to 1"
    assert (timTp['utc'        ] ==  1).all(), "Not all rows in timTp.utc are equal to 1"
    assert (timTp['raim'       ] ==  2).all(), "Not all rows in timTp.raim are equal to 2"
    assert (timTp['qErrInvalid'] ==  0).all(), "Not all rows in timTp.qErrInvalid are equal to 0"
    assert (timTp['TpNotLocked'] ==  0).all(), "Not all rows in timTp.TpNotLocked are equal to 0"
    assert (timTp['timeRefGnss'] == 15).all(), "Not all rows in timTp.timeRefGnss are equal to 15"
    assert (timTp['utcStandard'] ==  3).all(), "Not all rows in timTp.utcStandard are equal to  3"
    assert (timTp['towSubMS'   ] ==  0).all(), "Not all rows in timTp.towSubMS are equal to  0"
    timTp.drop(columns=['timeBase', 'utc', 'raim', 'qErrInvalid', 'TpNotLocked', 'timeRefGnss', 'utcStandard', 'towSubMS'], inplace=True)

    # Constants for time conversion
    gps_epoch = pd.Timestamp("1980-01-06 00:00:00", tz="UTC")
    leap_seconds = pd.Timedelta(seconds=18)  # current GPS-UTC offset (2025)

    # Vectorized conversion from GPS week and TOW to epoch seconds
    timTp["epochSec"] = (
        gps_epoch
        + pd.to_timedelta(timTp.week  * 7, unit="D" )
        + pd.to_timedelta(timTp.towMS    , unit="ms")
    )
    timTp.drop(columns=['week', 'towMS'], inplace=True)

    timTp['qErrFrac'] = timTp.qErr/1e12 # Convert qErr from picoseconds to seconds

    timTp['bn'] = baseName
    timTp['dut' ] = runData[baseName][chan]
    timTp['chan'] = chan
    return timTp

In [ ]:
dfs = []

dfs.append(readTimTp('baseline1', 'A'))
dfs.append(readTimTp('baseline2', 'A'))
dfs.append(readTimTp('baseline3', 'A'))
dfs.append(readTimTp('baseline4', 'A'))
dfs.append(readTimTp('baseline5', 'A'))

dfs.append(readTimTp('fixedPos1', 'A'))
dfs.append(readTimTp('fixedPos2', 'A'))

dfs.append(readTimTp('fixedL1ca1', 'A'))
dfs.append(readTimTp('fixedL1ca2', 'A'))

dfs.append(readTimTp('fixedL1l51', 'A'))
dfs.append(readTimTp('fixedL1l52', 'A'))
dfs.append(readTimTp('fixedL1l52', 'B'))
dfs.append(readTimTp('fixedL1l53', 'A'))
dfs.append(readTimTp('fixedL1l53', 'B'))

timTp = pd.concat(dfs, ignore_index=True)

In [ ]:
timTp

## Merge TIM-TP Data for Channels chA and chB by Time (Epoch Second)

In [ ]:
# Bring data from TIM-TP messages for each run together for each epoch second.
timTpA = timTp[timTp.chan == 'A'][['bn', 'epochSec', 'qErr', 'qErrFrac']]
timTpB = timTp[timTp.chan == 'B'][['bn', 'epochSec', 'qErr', 'qErrFrac']]

# Perform inner join on bn and epoch second
epSec = pd.merge(timTpA, timTpB, on=['bn', 'epochSec'], how='outer', suffixes=('A', 'B'))
#if timTpB.empty:
#    epSec.drop(columns=['qErrB'    ], inplace=True)
#    epSec.drop(columns=['qErrFracB'], inplace=True)

## Merge TICC Data and TIM-TP Data by Time (Epoch Second)

In [ ]:
# Merge TICC data from above with TIM-TP data on epoch second.
epSec = pd.merge(epSec, rcSec, on=['bn', 'epochSec'], how="outer")

In [ ]:
# Correct rcFrac with qErr
epSec['rcFracCorrA'] = epSec.rcFracA+epSec.qErrFracA

#if not timTpB.empty:
epSec['rcFracCorrB'] = epSec.rcFracB+epSec.qErrFracB
# Corrected phase difference between the two channels
epSec['rcFracCorrAB'] = epSec.rcFracCorrA - epSec.rcFracCorrB

## Count Per-Second Metrics for each Collection Run

In [ ]:
metrics = ['rcFracA', 'qErrA', 'qErrFracA', 'rcFracCorrA',
           'rcFracB', 'qErrB', 'qErrFracB', 'rcFracCorrB', 'rcFracCorrAB']
epSec.groupby('bn')[metrics].count()

## Optional Export to TimeLab

In [ ]:
# Select File->Import phase or frequency data fro ASCII file
#   Fill in Caption, Additional, Sampling Interval 1.0 sec, Input Frequency 1 Hz
#   Numeric Field 1 x 1.0 = Phase difference (sec), Data Format Decimal
expFracs = ['rcFracA', 'rcFracCorrA',
           'rcFracB', 'rcFracCorrB', 'rcFracCorrAB']
expDir = "exports"
for bn in epSec.bn.unique():
    for expFrac in expFracs:
        phaseErr = epSec[epSec.bn == bn][expFrac].dropna()
        if len(phaseErr) != 0:
            chan = 'foof'
            if expFrac[-2:] == 'AB':
                chan = f"{runData[bn]['A']}-{runData[bn]['B']}"
            elif expFrac[-1:] == 'A':
                chan = f"{runData[bn]['A']}"
            elif expFrac[-1:] == 'B':
                chan = f"{runData[bn]['B']}"
            else:
                assert False, f"Unexpected expFrac suffix {expFrac}"
            expFn = f"{expDir}/{bn}_{expFrac}_{chan}.txt"
            print(f"{expFn}")
            phaseErr.to_csv(expFn, index=False, header=False, float_format='%.15f')

In [ ]:
beg = 11000
end = 11500
bn = 'fixedL1l53'
plt.figure(figsize=(24, 6))
plt.plot(epSec[epSec.bn==bn].epochSec[beg:end], epSec[epSec.bn==bn].rcFracAB[beg:end]*1e9, marker='.', linestyle='-', color='b')
plt.plot(epSec[epSec.bn==bn].epochSec[beg:end], epSec[epSec.bn==bn].rcFracCorrAB[beg:end]*1e9, marker='.', linestyle='-', color='r')
plt.title(f"Phase Difference (epFracAB) vs Epoch Second for Run {bn}")
plt.xlabel('Epoch Second')
plt.ylabel('Phase Difference (epFracAB)')
plt.grid()
plt.show()


In [ ]:
# Compare callables for equality
def canonCallable(fn):
    g = inspect.unwrap(fn)               # peel decorators that use __wrapped__
    while isinstance(g, functools.partial):
        g = g.func                       # strip partial layers
    return getattr(g, "__func__", g)     # bound method → underlying function

def sameFunc(a, b):
    return canonCallable(a) is canonCallable(b)

In [ ]:
# Max averaging factor winLen: 3 for TDEV/MDEV, 2 for ADEV
def genTaus(n, thresh=25, winLen=3, perDecade=30):
    # Conservative τ_max so long-τ points still have usable statistics
    # Constants come from watching TimeLab behavior
    tauMax = n * 0.2 if winLen == 3 else n * 0.25

    # Expected max statistical point returned by the algorithm
    maxStat = (n-1) // winLen

    m_max_tau = int(np.floor(tauMax)) # XXXX Don't love this
    m_max = max(1, min(maxStat, m_max_tau))

    m_switch = max(1, int(np.floor(thresh)))
    denseTaus = np.arange(1, min(m_switch, m_max) + 1, dtype=int)

    if m_switch < m_max:
        # geometric spacing after the switch
        decades = max(1, int(np.ceil(np.log10(m_max) - np.log10(m_switch + 1))))
        m_geo = np.unique(np.rint(
            np.geomspace(m_switch + 1, m_max, decades * perDecade)
        ).astype(int))
        taus = np.unique(np.r_[denseTaus, m_geo])
    else:
        taus = denseTaus

    return taus

In [ ]:
# Adapters — Wrap allantools functions to match StatFn signature.
# Tip: use functools.partial to pre-set rate, data_type, taus, overlapping, etc.

# Type: any callable that takes your prepared data and returns (taus, values, errors_or_None)
StatFn = Callable[[np.ndarray], Tuple[np.ndarray, np.ndarray, Optional[np.ndarray]]]

# TimeLab uses OADEV, so let's not use the non-overlapping ADEV
#def wrap_adev(rate: float = 1.0, data_type: str = "phase", taus: Optional[Iterable[float]] = None, **kwargs) -> StatFn:
#    """
#    Returns a callable that takes 'data' and produces (taus, values, errors_or_None, n).
#    """
#    def _fn(data: np.ndarray) -> Tuple[np.ndarray, np.ndarray, Optional[np.ndarray]]:
#        # Many versions return a tuple: (taus, stat, stat_err, ns)
#        # If yours returns a dict, adjust the extraction below accordingly.
#        out = at.adev(data, rate=rate, data_type=data_type, taus=genTaus(len(data), winLen=2), **kwargs)
#        try:
#            tau, val, err, n = out  # common tuple form
#        except Exception:
#            assert False, "adev should return a tuple or dict-like form"
#            # Fallback for dict-like forms; update keys to match your version
#            tau = np.asarray(out.get("taus") or out.get("tau"))
#            val = np.asarray(out.get("adev") or out.get("adev") or out.get("values"))
#            err = out.get("adev_err") or out.get("adeverror") or None
#            if err is not None:
#                err = np.asarray(err)
#        return np.asarray(tau), np.asarray(val), (None if err is None else np.asarray(err)), n
#    return _fn
#
#adev = wrap_adev()

def wrap_oadev(rate: float = 1.0, data_type: str = "phase", taus: Optional[Iterable[float]] = None, **kwargs) -> StatFn:
    """
    Returns a callable that takes 'data' and produces (taus, values, errors_or_None, n).
    """
    def _fn(data: np.ndarray) -> Tuple[np.ndarray, np.ndarray, Optional[np.ndarray]]:
        out = at.oadev(data, rate=rate, data_type=data_type, taus=genTaus(len(data), winLen=2), **kwargs)
        try:
            tau, val, err, n = out  # common tuple form
        except Exception:
            assert False, "oadev should return a tuple or dict-like form"
            # Fallback for dict-like forms; update keys to match your version
            tau = np.asarray(out.get("taus") or out.get("tau"))
            val = np.asarray(out.get("oadev") or out.get("oadev") or out.get("values"))
            err = out.get("oadev_err") or out.get("adeverror") or None
            if err is not None:
                err = np.asarray(err)
        return np.asarray(tau), np.asarray(val), (None if err is None else np.asarray(err)), n
    return _fn

oadev = wrap_oadev()

def wrap_tdev(**kwargs) -> StatFn:
    def _fn(data: np.ndarray):
        out = at.tdev(data, rate=1.0, taus=genTaus(len(data), perDecade=30), **kwargs)
        try:
            tau, val, err, n = out
        except Exception:
            assert False, "tdev should return a tuple or dict-like form"
            tau = np.asarray(out.get("taus") or out.get("tau"))
            val = np.asarray(out.get("tdev") or out.get("values"))
            err = out.get("tdev_err") or None
            if err is not None:
                err = np.asarray(err)
        return np.asarray(tau), np.asarray(val), (None if err is None else np.asarray(err)), n
    return _fn

tdev = wrap_tdev()

def wrap_mtie(**kwargs) -> StatFn:
    def _fn(data: np.ndarray):
        out = at.mtie(data, rate=1.0, taus=genTaus(len(data), perDecade=30), **kwargs)
        try:
            tau, val, err, n = out
        except Exception:
            assert False, "mtie should return a tuple or dict-like form"
            tau = np.asarray(out.get("taus") or out.get("tau"))
            val = np.asarray(out.get("mtie") or out.get("values"))
            err = out.get("mtie_err") or None
            if err is not None:
                err = np.asarray(err)
        return np.asarray(tau), np.asarray(val), (None if err is None else np.asarray(err)), n
    return _fn

mtie = wrap_mtie()

In [ ]:
def statPlotStart(
    statFn: Optional[StatFn] = tdev,
    title: Optional[str] = None,
    xlabel: str = r"$\tau$",
    ylabel: Optional[str] = None,
    logx: bool = True,
    logy: bool = True,
    grid: bool = True,
    figsize: Tuple[float, float] = (16, 9),
) -> Tuple[plt.Figure, plt.Axes]:

    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize=figsize)
    if logx and logy:
        ax.set_xscale("log")
        ax.set_yscale("log")
    elif logx:
        ax.set_xscale("log")
    elif logy:
        ax.set_yscale("log")
    if title:
        ax.set_title(title, fontsize=20)
    ax.set_xlabel(xlabel, fontsize=20)
    if ylabel is None:
        if sameFunc(statFn, tdev):
            ylabel = r"$\sigma_x(\tau)$"
        elif sameFunc(statFn, oadev):
            ylabel = r"$\sigma_y(\tau)$"
        elif sameFunc(statFn, mtie):
            ylabel = r"MTIE($\tau$)"
    ax.set_ylabel(ylabel, fontsize=20)
    if grid:
        ax.grid(True, which="both", alpha=0.35)

    return fig, ax

In [ ]:
def statPlotTrace(
    ax: plt.Axes,
    s: pd.Series,
    statFn: StatFn,
    secLimit: Optional[int] = sys.maxsize,
    *,
    label: Optional[str] = None,
    errBars: bool = False,
    # Matplotlib styling (pass whatever you like; e.g., color, linestyle, marker, alpha, linewidth ...)
    **line_kwargs,
) -> Tuple[np.ndarray, np.ndarray, Optional[np.ndarray]]:
    """
    Preprocesses the Series, calls statFn, then plots (taus, value) on ax.
    Returns (taus, values, errors_or_None) for further use if needed.
    """
    taus, vals, errs, n = statFn(s[:secLimit].dropna())  # Apply the statistic function to the Series, limited by secLimit

    mult = 1e9 if sameFunc(statFn, tdev) or sameFunc(statFn, mtie) else 1.0  # Convert to nanoseconds if using TDEV
    line = plt.loglog(taus, mult*vals, label=label, **line_kwargs)[0]
    color = line.get_color()

    if errBars:
        goodN = n >= 10 # Ensure we have enough data points for the statistic to be meaningful
        ax.errorbar(taus[goodN], mult*vals[goodN], yerr=mult*errs[goodN] if errs is not None else None, fmt='none', color=color, elinewidth=1.4,       # “web” thickness
                capsize=2,            # “flange” length
                capthick=1.4, **line_kwargs)
    return taus, vals, errs

In [ ]:

def statPlotFinish(
    ax: plt.Axes,
    statFn: Optional[StatFn] = tdev,
    legend: bool = True,
    legend_loc: str = "best",
    tight_layout: bool = False,
):
    if legend:
        ax.legend(loc=legend_loc, fontsize=18)
    if tight_layout:
        ax.figure.tight_layout()
    ax.tick_params(axis='both', which='both', labelsize=14)

    ax.grid(which="major", linestyle="-", linewidth=1.0, color="gray")
    ax.grid(which="minor", linestyle="--", linewidth=1.0, color="lightgray")

    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:g} s"))

    base_font = float(plt.rcParams.get("font.size", 10))
    annotation_font = max(base_font * 0.85, 10)
    box_font = max(base_font * 0.9, 10)

    if sameFunc(statFn, tdev) or sameFunc(statFn, mtie):
        def _format_ns(value: float, _tick: int) -> str:
            if value <= 0:
                return "XXX"
            if value >= 1:
                return f"{value:.0f} ns"
            if value >= 1e-3:
                return f"{1e3*value:3.0f} ps"
            return f"D{value:.1e} ns"

        ns_formatter = FuncFormatter(_format_ns)
        ax.yaxis.set_major_formatter(ns_formatter)
        ax.yaxis.set_minor_formatter(ns_formatter)
        #ax.yaxis.set_major_locator(MaxNLocator(nbins=10, prune='lower', min_n_ticks=5)) # AI suggestion
        #ax.yaxis.set_minor_locator(MaxNLocator(5))

        ymin, ymax = ax.get_ylim()
        if ymin < 0.1:
            yShadeMin = ymin
            yShadeMax = 0.1
            ax.axhspan(yShadeMin, yShadeMax, color="lightgray", alpha=0.3)

            y_center = np.sqrt(yShadeMin * yShadeMax)
            x_limits = ax.get_xlim()
            x_center = np.sqrt(x_limits[0] * x_limits[1])
            ax.text(
                x_center,
                y_center,
                "RMS Jitter Floor of TAPR TICC",
                ha="center",
                va="center",
                fontsize=annotation_font,
                color="black",
                bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"),
            )
            ax.set_ylim(ymin, ymax)

    ax.text(
        0.95,
        0.1,
        "Instrument: TAPR TICC\nReference: Geppetto GPSDO with OH300 5ppb OCXO",
        ha="right",
        va="bottom",
        multialignment="left",
        transform=ax.transAxes,
        fontsize=box_font,
        bbox=dict(
            facecolor="white",
            edgecolor="black",
            boxstyle="round,pad=0.3",
        ),
    )


In [ ]:
fig, ax = statPlotStart(title = r"Impact of u-blox F9T $\mathtt{qErr}$ Corrections on TDEV($\tau$)")

statPlotTrace(ax, epSec[epSec.bn=='fixedPos1'].rcFracA    , tdev, secLimit=200000, label='Uncorrected', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedPos1'].rcFracCorrA, tdev, secLimit=200000, label='Corrected'  , linestyle='-', linewidth=2)
statPlotFinish(ax)

In [ ]:
fig, ax = statPlotStart(mtie, title = r"Impact of u-blox F9T $\mathtt{qErr}$ Corrections on MTIE($\tau$)")

statPlotTrace(ax, epSec[epSec.bn=='fixedPos1'].rcFracA    , mtie, secLimit=200000, label='Uncorrected', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedPos1'].rcFracCorrA, mtie, secLimit=200000, label='Corrected'  , linestyle='-', linewidth=2)
statPlotFinish(ax, mtie)

In [ ]:
fig, ax = statPlotStart(oadev, title = r"Impact of u-blox F9T $\mathtt{qErr}$ Corrections on ADEV($\tau$)")

statPlotTrace(ax, epSec[epSec.bn=='fixedPos1'].rcFracA    , oadev, secLimit=200000, label='Uncorrected', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedPos1'].rcFracCorrA, oadev, secLimit=200000, label='Corrected'  , linestyle='-', linewidth=2)
statPlotFinish(ax, oadev)

In [ ]:
fig, ax = statPlotStart(title = r"Comparing TDEV for Two u-blox F9Ts with Common Anntenna and $\mathtt{qErr}$ Corrections Individually and to Each Other")

statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA,  tdev, secLimit=300000, label='F9T-Bob'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrB,  tdev, secLimit=300000, label='F9T-PT'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrAB, tdev, secLimit=300000, label='F9T-Bob vs F9T-PT'  , linestyle='-', linewidth=2)
statPlotFinish(ax)

In [ ]:
fig, ax = statPlotStart(title = r"Comparing MTIE for Two u-blox F9Ts with Common Anntenna and $\mathtt{qErr}$ Corrections Individually and to Each Other")

statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA,  mtie, secLimit=30000, label='F9T-Bob'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrB,  mtie, secLimit=30000, label='F9T-PT'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrAB, mtie, secLimit=30000, label='F9T-Bob vs F9T-PT'  , linestyle='-', linewidth=2)
statPlotFinish(ax, mtie)

In [ ]:
fig, ax = statPlotStart(title = r"Comparing ADEV for Two u-blox F9Ts with Common Anntenna and $\mathtt{qErr}$ Corrections Individually and to Each Other")

statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA,  oadev, secLimit=30000, label='F9T-Bob'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrB,  oadev, secLimit=30000, label='F9T-PT'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrAB, oadev, secLimit=30000, label='F9T-Bob vs F9T-PT'  , linestyle='-', linewidth=2)
statPlotFinish(ax, oadev)

In [ ]:
fig, ax = statPlotStart(title = r"Comparing TDEV for u-blox F9T and Meinberg M600 DHQ")

statPlotTrace(ax, epSec[epSec.bn=='ctiCns1'   ].rcFracB,     tdev, secLimit=300000, label='Jittery CNS Clock II'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='f9tM600-2' ].rcFracB,     tdev, secLimit=300000, label='M600 DHQ'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracA    , tdev, secLimit=300000, label='Uncorrected F9T-Bob', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA, tdev, secLimit=300000, label='Corrected F9T-Bob'  , linestyle='dotted', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracB    , tdev, secLimit=300000, label='Uncorrected F9T-PT', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrB, tdev, secLimit=300000, label='Corrected F9T-PT'  , linestyle='dotted', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='ctiCns1'   ].rcFracA,     tdev, secLimit=300000, label='Cheap CTI OCXO'  , linestyle='-', linewidth=2)
statPlotFinish(ax)

In [ ]:
fig, ax = statPlotStart(title = r"Comparing MTIE for u-blox F9T and Meinberg M600 DHQ")

statPlotTrace(ax, epSec[epSec.bn=='ctiCns1'   ].rcFracB,     mtie, secLimit=300000, label='Jittery CNS Clock II'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='f9tM600-2' ].rcFracB,     mtie, secLimit=300000, label='M600 DHQ'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracA    , mtie, secLimit=300000, label='Uncorrected F9T-Bob', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA, mtie, secLimit=300000, label='Corrected F9T-Bob'  , linestyle='dotted', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracB    , mtie, secLimit=300000, label='Uncorrected F9T-PT', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrB, mtie, secLimit=300000, label='Corrected F9T-PT'  , linestyle='dotted', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='ctiCns1'   ].rcFracA,     mtie, secLimit=300000, label='Cheap CTI OCXO'  , linestyle='-', linewidth=2)
statPlotFinish(ax, mtie)

In [ ]:
fig, ax = statPlotStart(title = r"Comparing ADEV for u-blox F9T and Meinberg M600 DHQ")

statPlotTrace(ax, epSec[epSec.bn=='ctiCns1'   ].rcFracB,     oadev, secLimit=300000, label='Jittery CNS Clock II'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='f9tM600-2' ].rcFracB,     oadev, secLimit=300000, label='M600 DHQ'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracA    , oadev, secLimit=300000, label='Uncorrected F9T-Bob', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA, oadev, secLimit=300000, label='Corrected F9T-Bob'  , linestyle='dotted', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracB    , oadev, secLimit=300000, label='Uncorrected F9T-PT', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrB, oadev, secLimit=300000, label='Corrected F9T-PT'  , linestyle='dotted', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='ctiCns1'   ].rcFracA,     oadev, secLimit=300000, label='Cheap CTI OCXO'  , linestyle='-', linewidth=2)
statPlotFinish(ax, oadev)

In [ ]:
fig, ax = statPlotStart(title = r"Comparing TDEV u-blox F9T as Config is Improved for Timing Performance")

statPlotTrace(ax, epSec[epSec.bn=='baseline5' ].rcFracA,     tdev, secLimit=300000, label='Factory Reset, Config Cleared'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1ca1'].rcFracA    , tdev, secLimit=300000, label='Fixed Position L1 C/A, Run 1', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracA    , tdev, secLimit=300000, label='Fixed Position L1 C/A & L5, Run 3', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA, tdev, secLimit=300000, label='Fixed Position L1 C/A & L5, qErr Corrected'  , linestyle='dotted', linewidth=2)
statPlotFinish(ax)

In [ ]:
fig, ax = statPlotStart(title = r"Comparing MTIE u-blox F9T as Config is Improved for Timing Performance")

statPlotTrace(ax, epSec[epSec.bn=='baseline5' ].rcFracA,     mtie, secLimit=300000, label='Factory Reset, Config Cleared'  , linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1ca1'].rcFracA    , mtie, secLimit=300000, label='Fixed Position L1 C/A, Run 1', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracA    , mtie, secLimit=300000, label='Fixed Position L1 C/A & L5, Run 3', linestyle='-', linewidth=2)
statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA, mtie, secLimit=300000, label='Fixed Position L1 C/A & L5, qErr Corrected'  , linestyle='dotted', linewidth=2)
statPlotFinish(ax, mtie)

In [ ]:

presentation_fonts = {
    "font.size": 20,
    "axes.titlesize": 24,
    "axes.labelsize": 20,
    "legend.fontsize": 18,
    "xtick.labelsize": 18,
    "ytick.labelsize": 18,
}

with plt.rc_context(presentation_fonts):
    fig, ax = statPlotStart(title = r"Comparing ADEV u-blox F9T as Config is Improved for Timing Performance")

    statPlotTrace(ax, epSec[epSec.bn=='baseline5' ].rcFracA,     oadev, secLimit=300000, label='Factory Reset, Config Cleared'  , linestyle='-', linewidth=2)
    statPlotTrace(ax, epSec[epSec.bn=='fixedL1ca1'].rcFracA    , oadev, secLimit=300000, label='Fixed Position L1 C/A, Run 1', linestyle='-', linewidth=2)
    statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracA    , oadev, secLimit=300000, label='Fixed Position L1 C/A & L5, Run 3', linestyle='-', linewidth=2)
    statPlotTrace(ax, epSec[epSec.bn=='fixedL1l53'].rcFracCorrA, oadev, secLimit=300000, label='Fixed Position L1 C/A & L5, qErr Corrected'  , linestyle='dotted', linewidth=2)
    statPlotFinish(ax, oadev, tight_layout=True)
    ax.tick_params(axis='both', which='both', labelsize=presentation_fonts["xtick.labelsize"])
    fig


In [ ]:
# XXX Next:
# . Clear output and checkpoint to git
# . Try ADEV
# . Try MTIE
# . Plot M600
# . Compare to baseline 5 without fixed position
# . Confirm export to Timelab produces comparable plots
# . Can you get error bars on TDEV and other plots?
# Collecte data on AliExpress OCXO
# Try overlaid TDEV and MTIE with gradient shading between them
# See exactly what Taitien oscillator is on mini PT
# Review other devices in talk proposal for data collection
# Collect data on CNS Clock
# Study time error histograms more carefully
# Get deeper understanding of different deviations
# Comapre ADEV to other reported results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Normal distribution parameters (ns)
mu_ns = 0.0
sigma_ns = 15.0  # 15 ns RMS

# X range and PDF
x = np.linspace(mu_ns - 6*sigma_ns, mu_ns + 6*sigma_ns, 2000)
pdf = (1.0 / (np.sqrt(2*np.pi) * sigma_ns)) * np.exp(-0.5 * ((x - mu_ns)/sigma_ns)**2)

# Plot
#plt.figure(figsize=(24, 6))
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(x, pdf, lw=2)

# Highlight 200 ps-wide (0.2 ns) vertical bands centered at 0, ±15 ns
half_width_ns = 0.1  # ±100 ps
for center in (0.0, -15.0, 15.0):
    ax.axvspan(center - half_width_ns, center + half_width_ns, alpha=0.75)

ax.set_xlabel("Time error (ns)", fontsize=24)
ax.set_ylabel("Probability density (1/ns)", fontsize=24)
ax.set_title("Normal(μ=0, σ=15 ns) with 200 ps-wide bands at 0 and ±15 ns", fontsize=32)
ax.set_xlim(mu_ns - 5*sigma_ns, mu_ns + 5*sigma_ns)
ax.grid(True, alpha=0.3)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim(-50, 50)
plt.show()


In [ ]:
step_sigma_ns = 1.0  # Standard deviation of each step; tweak to control walk volatility
n_steps = 500
seed = None  # Set to an int (e.g., 42) for reproducible walks

rng = np.random.default_rng(seed)

def random_walk(start_ns: float, step_sigma: float, steps: int, *, generator: np.random.Generator) -> np.ndarray:
    """Return a random walk (in ns) that begins at start_ns."""
    increments = generator.normal(loc=0.0, scale=step_sigma, size=steps)
    return start_ns + np.concatenate(([0.0], np.cumsum(increments)))

walk_neg = random_walk(-5.0, step_sigma_ns, n_steps, generator=rng)
walk_pos = random_walk(10.0, step_sigma_ns, n_steps, generator=rng)
step_index = np.arange(n_steps + 1)

plt.figure(figsize=(12, 6))
plt.plot(step_index, walk_neg, label='Walk starting at -5 ns', linewidth=2)
plt.plot(step_index, walk_pos, label='Walk starting at +10 ns', linewidth=2)
plt.xlabel('Step')
plt.ylabel('Current Value (ns)')
plt.title('Random Walks with Adjustable Step Size')
plt.legend()
plt.grid(True, alpha=0.35)
plt.tight_layout()


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

seconds = np.linspace(0, 86_400, 86_401)

def utc_offset_ns(t: np.ndarray) -> np.ndarray:
    """Piecewise linear UTC offset (ns) for the evil GNSS clock."""
    offsets = np.empty_like(t)

    night_mask = (t < 21_600)
    offsets[night_mask] = -2.0 * t[night_mask]

    morning_mask = (t >= 21_600) & (t < 43_200)
    offsets[morning_mask] = -43_200.0 + 2.0 * (t[morning_mask] - 21_600.0)

    afternoon_mask = (t >= 43_200) & (t < 64_800)
    offsets[afternoon_mask] = 2.0 * (t[afternoon_mask] - 43_200.0)

    evening_mask = t >= 64_800
    offsets[evening_mask] = 43_200.0 - 2.0 * (t[evening_mask] - 64_800.0)

    return offsets

offset_ns = utc_offset_ns(seconds)
offset_us = offset_ns / 1_000.0
hours = seconds / 3_600.0

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(hours, offset_us, linewidth=2, color='crimson')

ax.set_xticks([0, 6, 12, 18, 24])
ax.set_xticklabels(['12 AM', '6 AM', '12 PM', '6 PM', '12 AM'])

ax.set_xlabel('Time of Day', fontsize=24)
ax.set_ylabel('Offset from UTC (µs)', fontsize=24)
ax.set_title('Evil GNSS Clock: Daily UTC Offset Profile', fontsize=32)
ax.axhline(0, color='black', linewidth=1, linestyle='--', alpha=0.6)
ax.grid(True, which='both', alpha=0.35)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
